In [3]:
## Download BedMachine data directly from  NSIDC

from earthdata import Auth, DataCollections, DataGranules, Store

In [29]:
# Set working directories (local)

data_loc = "/home/jovyan/data/"

In [10]:
# Authenticate Earthdata credentials

auth = Auth().login(strategy='netrc')
if auth.authenticated is False:
    auth = Auth().login(strategy='interactive')

You're now authenticated with NASA Earthdata Login


In [19]:
# Identify data collections available based on keyword (here, use "bed topography" to identify collections with bed topography (e.g., BedMachine)
Query = DataCollections().keyword('bed topography').provider("NSIDC_ECS")

print(f'Collections found: {Query.hits()}')

Collections found: 6


In [20]:
# Review the shortName and Abstract for all collections in order to identify the best dataset to use.
collections = Query.fields(['ShortName','Abstract']).get()
print(collections)

[{
  "meta": {
    "concept-id": "C2050907241-NSIDC_ECS",
    "granule-count": 1,
    "provider-id": "NSIDC_ECS"
  },
  "umm": {
    "ShortName": "IDBMG4",
    "Abstract": "This data set contains a bed topography/bathymetry map of Greenland based on mass conservation, multi-beam data, and other techniques. It also includes surface elevation and ice thickness data, as well as an ice/ocean/land mask."
  }
}, {
  "meta": {
    "concept-id": "C1946898764-NSIDC_ECS",
    "granule-count": 1,
    "provider-id": "NSIDC_ECS"
  },
  "umm": {
    "ShortName": "NSIDC-0756",
    "Abstract": "This data set, part of the NASA Making Earth System Data Records for Use in Research Environments (MEaSUREs) program, contains a bed topography/bathymetry map of Antarctica based on mass conservation, streamline diffusion, and other methods. The data set also includes ice thickness, surface elevation, an ice/ocean/land mask, ice thickness estimation errors, and a map showing where each method was utilized."
  }

In [27]:
# For MEaSUREs BedMachine data, concept-id: C1664160168-NSIDC_ECS
bedmachine = 'C1664160168-NSIDC_ECS'

# TODO: Define a bounding box from a IMBIE / MEaSUREs boundary
# Query = DataGranules().concept_id(bedmachine).bounding_box(-134.7,58.9,-133.9,59.2)

# For testing, download without a bounding box
Query = DataGranules().concept_id(bedmachine)

print(f'Granules found: {Query.hits()}')

Granules found: 1


In [28]:
# Get the relevant granules for the collectin
granules = Query.get()
print(granules)

[Collection: {'EntryTitle': 'MEaSUREs BedMachine Antarctica V001'}
Spatial coverage: {'HorizontalSpatialDomain': {'Geometry': {'BoundingRectangles': [{'WestBoundingCoordinate': -180.0, 'EastBoundingCoordinate': 180.0, 'NorthBoundingCoordinate': -53.0, 'SouthBoundingCoordinate': -90.0}]}}}
Temporal coverage: {'RangeDateTime': {'BeginningDateTime': '1970-01-01T00:00:01.000Z', 'EndingDateTime': '2019-10-01T23:59:59.000Z'}}
Size(MB): 795.317
Data: ['https://n5eil01u.ecs.nsidc.org/DP4/MEASURES/NSIDC-0756.001/1970.01.01/BedMachineAntarctica_2019-11-05_v01.nc']]


In [26]:
# Download the data! 

# How long does this take?
%%time

# Access the data from on-prem at NSIDC. Store this at "data_loc" locally.
access = Store(auth)
files = access.get(granules, local_path = data_loc)

 Getting 1 granules, approx download size: 0.78 GB


SUBMITTING | :   0%|          | 0/1 [00:00<?, ?it/s]

PROCESSING | :   0%|          | 0/1 [00:00<?, ?it/s]

COLLECTING | :   0%|          | 0/1 [00:00<?, ?it/s]

CPU times: user 1.73 s, sys: 1.84 s, total: 3.57 s
Wall time: 57.3 s
